In [1]:
import os
import sys

sys.path.insert(0,'src')
sys.path.insert(0,'../src')

from qwertyfolio import Asset, Transaction, PortfolioManager
from qwertyfolio.brokerage import OurBroker

### setup
define data files and clear data for new portfolio

In [2]:
port_file = 'portfolio.json'
transaction_file = 'transaction_log.csv' 
brokerage_stub = OurBroker()
portfolio = PortfolioManager(
    port_file, 
    transaction_file,
    brokerage=brokerage_stub,
    use_brokerage_holdings=True,
    use_brokerage_transactions=True
    )
portfolio.clear_log()
portfolio.update_cash_balance(500000)


Getting positions from broker API


## define transactions to test 
exported from a spreadsheet as csv, then mutated to structure ...
{ date: [ { tx_data: dict }, ... ]}
``

In [3]:
expected_result = 4800
trades: dict[str:list[dict]] = {
    "2024-12-12 10:00:00": [
        {
            "symbol": "XYZ",
            "quantity": 120,
            "price": 100
        },
    ],
    "2024-12-20 10:00:00": [
        {
            "symbol": "XYZ",
            "quantity": 120,
            "price": 80
        },
    ]
}


In [5]:
for date in trades.keys():
    legs = [ Asset(**adict)  for adict in trades[date] ]
    tx = Transaction(timestamp=date, legs=legs)
    # print(tx)
    if date == '2024-12-12 10:00:00':
        portfolio.execute_open(tx)
    elif date == '2025-02-21 10:00:00':
        portfolio.execute_close(tx)
    else:
        portfolio.execute_transaction(tx)


portfolio.print_portfolio()
pnl: float = portfolio.calculate_pnl({"XYZ": 110, "DELL": 5.01})

pnl


____ Current Portfolio ____
Cash Balance: $456800.00
+----+----------+------------+---------+----------------------+--------------+---------------------+--------------+----------------------+--------------+----------------+---------+---------+---------+----------------------------+--------------+-----------+--------------+
|    | symbol   |   quantity |   price |   average_open_price | asset_type   | underlying_symbol   |   multiplier | days_to_expiration   | expires_at   | strike_price   | delta   | gamma   | theta   | quote_date                 | order_type   |   chainid |   roll_count |
|----+----------+------------+---------+----------------------+--------------+---------------------+--------------+----------------------+--------------+----------------+---------+---------+---------+----------------------------+--------------+-----------+--------------|
|  0 | DELL     |         42 |    1.99 |                 1.99 | S            | DELL                |            1 |                

9741.240000000009